# Traveling Salesman Problem
### Definition
<p>
    Given a collection of cities, the traveling salesman must determine the
    shortest route that will enable him to visit each city precisely once and
    then return back to his starting point.
    The distance between each city is given and is assumed to be the same in
    both directions.
    Objective is to minimize the total distance to be travelled.
<p/>

<p>
    This type of problem frequently occurs when coding the AI for strategy
    games. Often it’s necessary to create the shortest path for a unit that
    will start at one waypoint, end at another, and pass through several
    predefined areas along the way, to pick up resources, food, energy, and so
    on. It can also be used as part of the route planning AI for a Quake-like
    FPS bot.
    Some of the landmarks in TSP solving, starting from the 1950s.
</p>

In [124]:
import numpy as np
from copy import deepcopy

In [125]:
def permutation_crossover(parent1, parent2, crossover_rate):
    
    if np.random.rand() < crossover_rate:
        return deepcopy(parent1), deepcopy(parent2)
    
    chromosone_length = len(parent1)
    # random number between [0, chromosone_length)
    cp1 = np.random.randint(0, chromosone_length) 
    cp2 = np.random.randint(0, chromosone_length)
    #cp1, cp2 = 3, 6

    while cp1 == cp2:
        cp2 = np.random.randint(0, chromosone_length)
    
    child1, child2 = deepcopy(parent1), deepcopy(parent2)
    value_to_map1 = child1[cp1:cp2]
    value_to_map2 = child2[cp1:cp2]
    
    for value1, value2 in zip(value_to_map1, value_to_map2):
        for i, city in enumerate(child1):       
            if city == value1:
                child1[i] = value2
            elif city == value2:
                child1[i] = value1
        
        for i, city in enumerate(child2):
            if city == value1:
                child2[i] = value2
            elif city == value2:
                child2[i] = value1
                
    return child1, child2

In [126]:
def insertion_mutation(chromosone, mutation_rate):
    
    if np.random.rand() < mutation_rate:
        return chromosone
    
    chromosone_length = len(chromosone)
    selected_index = np.random.randint(0, chromosone_length)
    insertion_index = np.random.randint(0, chromosone_length)
#     selected_index, insertion_index = 2, 5
    while selected_index == insertion_index:
        insertion_index = np.random.randint(0, chromosone_length)
    
    gene = chromosone[selected_index]
    step = -1 if insertion_index < selected_index else 1 # insertion_index > selected_index
    
    for i in range(selected_index, insertion_index, step):
            chromosone[i] = chromosone[i+step]
    
    chromosone[insertion_index] = gene      

    return chromosone

In [127]:
def find_cost(cost_matrix, individual):
    stop = len(individual)-1
    cost=0
    for i in range(stop):
        x, y = individual[i], individual[i+1]
        cost += cost_matrix[x, y]
    return cost

In [128]:
def compute_fitness_score(cost_matrix, population):
    fitness = np.zeros(population.shape[0])
    for i, individual in enumerate(population):
        fitness[i] = find_cost(cost_matrix, individual)
    fitness = fitness.max() - fitness
    return fitness

In [129]:
def sigma_scaling(fitness_score):
    average = fitness_score.mean()
    std_2 = 2 * fitness_score.std()
    new_fitness = (fitness_score - average) / std_2
    return new_fitness

In [130]:
def tournament_selection(population, fitness_score, k):
    tournament = np.empty(k, dtype=int)
    pop_size = population.shape[0] 
    for i in range(k):
        random_no = np.random.randint(0, pop_size)
        while random_no in tournament:
            random_no = np.random.randint(0, pop_size)
        tournament[i] = random_no

    best = max(tournament, key=lambda x: fitness_score[x])

    return best

In [131]:
def mating(population, fitness_score, crossover_rate, mutation_rate):
    pop_size = population.shape[0]
    new_population = np.empty(population.shape, dtype=int)
    k = 2 # number of individual for tournament selection
    for i in range(0, pop_size, 2):
        idx_1 = tournament_selection(population, fitness_score, k)
        idx_2 = tournament_selection(population, fitness_score, k)
        parent1, parent2 = population[idx_1], population[idx_2]
        child1, child2 = permutation_crossover(parent1, parent2, crossover_rate)
        child1 = insertion_mutation(child1, mutation_rate)
        child2 = insertion_mutation(child2, mutation_rate)
        new_population[i] = child1
        new_population[i+1] = child2
        
    return new_population

In [132]:
# Not yet working properly 
def initialize_population(pop_size, number_cities):
    population = np.empty((pop_size, number_cities), dtype=int)
    
    for i in range(pop_size):
        city = np.empty(number_cities, dtype=int)
        flag=0
        for j in range(number_cities):
            flag +=1
            rand_no = np.random.randint(0, number_cities)
            while rand_no in city and flag !=number_cities:
                rand_no = np.random.randint(0, number_cities)
#                 print(rand_no, city)    
                print(flag)
            city[j] = rand_no
            

        population[i] = city
        print(f'Individual {i} created...')
    return population

In [133]:
# Parameters
# total_cities = 6
# population_size = np.math.factorial(total_cities)
# population_size = 50
epochs = 1_000
show_info_after = 100
crossover_rate = 0.7
mutation_rate = 0.001

population = np.array([
    [1, 2, 3, 5, 4, 0],
    [3, 1, 2, 4, 0, 5],
    [5, 1, 2, 3, 0, 4],
    [0, 1, 5, 4, 3, 2],
    [1, 3, 0, 4, 5, 2],
    [0, 5, 3, 4, 1, 2]
])
population_size = population.shape[0]
total_cities = population.shape[1]

In [134]:
# This costs matrix show the distance between each city 
# City are 0, 1, 2, 3, 4, 5
costs = np.array([
    [  0, 105,  95, 340, 256, 292],
    [105,   0, 372, 288, 145, 309],
    [ 95, 372,   0, 155, 200, 188],
    [340, 288, 155,   0, 244, 327],
    [256, 145, 200, 244,   0,  98],
    [292, 309, 188, 327,  98,   0]
])

In [135]:
# Example: Distance between city 2 and city 5
print(f'Distance between city 2 and city 5 is : {costs[2, 5]}')
# Example: Distance between city 0 and city 4
print(f'Distance between city 0 and city 4 is : {costs[0, 4]}')
costs

Distance between city 2 and city 5 is : 188
Distance between city 0 and city 4 is : 256


array([[  0, 105,  95, 340, 256, 292],
       [105,   0, 372, 288, 145, 309],
       [ 95, 372,   0, 155, 200, 188],
       [340, 288, 155,   0, 244, 327],
       [256, 145, 200, 244,   0,  98],
       [292, 309, 188, 327,  98,   0]])

In [136]:
fitness_score = compute_fitness_score(costs, population)
fitness_score

array([224.,  24.,   0., 521., 262.,  52.])

In [137]:
# Running Genetic Algorithm
print('Start running Genetic Algorithm')
for i in range(1, epochs+1, 1):
    population = mating(population, fitness_score, crossover_rate, mutation_rate)
    fitness_score = compute_fitness_score(costs, population)
    if i % show_info_after == 0 and i != 0:
        print(f'Epoch {i} done...')

Start running Genetic Algorithm
Epoch 100 done...
Epoch 200 done...
Epoch 300 done...
Epoch 400 done...
Epoch 500 done...
Epoch 600 done...
Epoch 700 done...
Epoch 800 done...
Epoch 900 done...
Epoch 1000 done...


In [138]:
fitness_score

array([594.,  99., 448., 342.,   0., 788.])